# DHL Optimization
### 1. Dataset Set up
Generate variable cost matrices (shipping cost from supplier to warehouse, shipping cost from warehouse to customer, variable cost of supplier and variable cost of warehouse), and fixed cost matrices (fixed cost of supplier and fixed cost of warehouse).

In [1]:
# Import package
import pandas as pd
import numpy as np
import scipy
import scipy.optimize as optimize

In [2]:
# Read data
xls = pd.ExcelFile('/Users/JYang/Downloads/Network.xlsm')

In [3]:
# Suppliers to Warehouse Rate 
s_wh_r = pd.read_excel(xls, 'Supplier-WHs-Rates')
s_wh_r = s_wh_r[4:]

# Change Column Name
new_header_swh = s_wh_r.iloc[0]
s_wh_r = s_wh_r[1:]
s_wh_r.columns = new_header_swh

# Reset Index
s_wh_r.reset_index(drop=True, inplace=True)

# Take a look
s_wh_r.head(5)

4,to/from,"Newark, NJ","Atlanta, GA","Dallas, TX","Seattle, WA","Los Angeles, CA","Chicago, IL","Columbus, OH","Denver, CO","Oakland, CA","Lakeland, FL","Phoenix, AZ"
0,Houston TX,4037.5,1980,597.5,6100,3862.5,2715,2885,2579.75,4774.5,2443,2934.75
1,Phoenix AZ,6130,4620,2680,3777.5,930,4505,4807.25,2265.25,1839.5,5379,250
2,San Jose CA,7307.5,6072.5,4222.5,2110,847.5,5392.5,6178.25,3256.25,250,6972,1773.75
3,Jacksonville FL,2325,862.5,2482.5,7552.5,6037.5,2680,2025.75,4352.5,6943,486.25,5103.25
4,Columbus OH,1310,1417.5,2602.5,6050,5620,892.5,250,3127.5,6103.5,2501,4800


In [4]:
# Convert dataframe to matrix
s_wh_r_matrix = s_wh_r.loc[:, s_wh_r.columns != 'to/from']
s_wh_r_matrix = s_wh_r_matrix.to_numpy()

# Take a look
s_wh_r_matrix.shape

(10, 11)

In [5]:
# Warehouse to Customer Rate 
wh_c_r = pd.read_excel(xls, 'WHs-Customers-Rates')
wh_c_r = wh_c_r[4:]

# Change Column Name
new_header_whc = wh_c_r.iloc[0]
wh_c_r = wh_c_r[1:]
wh_c_r.columns = new_header_whc

# Reset Index
wh_c_r.reset_index(drop=True, inplace=True)

# Take a look
wh_c_r.head(5)

4,to/from,"Newark, NJ","Atlanta, GA","Dallas, TX","Seattle, WA","Los Angeles, CA","Chicago, IL","Columbus, OH","Denver, CO","Oakland, CA","Lakeland, FL","Phoenix, AZ"
0,New York NY,8.5,16.5,23.5,36.5,36.5,15.5,13.5,25.5,37.5,19.5,32.5
1,Los Angeles CA,35.5,29.5,22.5,19.5,7.5,28.5,30.5,18.5,11.5,33.5,11.5
2,Chicago IL,15.5,15.5,17.5,28.5,28.5,7.5,11.5,18.5,29.5,19.5,25.5
3,Miami FL,20.5,14.5,21.5,41.5,35.5,21.5,19.5,28.5,38.5,10.5,31.5
4,Dallas TX,23.5,15.5,7.5,29.5,22.5,17.5,18.5,15.5,25.5,19.5,18.5


In [6]:
# Convert dataframe to matrix
wh_c_r_matrix = wh_c_r.loc[:987, wh_c_r.columns != 'to/from']
wh_c_r_matrix = wh_c_r_matrix.to_numpy()

# Take a look
wh_c_r_matrix.shape

(988, 11)

In [7]:
# Supplier Cost Data
supplier = pd.read_excel(xls, 'Suppliers')

In [8]:
# Supplier Fixed Cost Data
s_fc = supplier.loc[:, 'Fixed Cost']

# Convert to list
s_fc = s_fc.tolist()

In [9]:
# Supplier Variable Cost per Shipment Data
s_vcps = supplier.loc[:, 'Variable Cost / Shipment']

# Convert to list
s_vcps = s_vcps.tolist()

all_s_vpcps = []
for i in range(10):
    all_s_vpcps.append(np.full(11, s_vcps[i]).tolist())
s_vpcps = all_s_vpcps

In [10]:
# Warehouse Cost Data
wh = pd.read_excel(xls, 'WHs')

In [11]:
# Warehouse Fixed Cost Data
wh_fc = wh.loc[:, 'Fixed Cost']

# Convert to list
wh_fc = wh_fc.tolist()

In [12]:
# Demand Data
cust = pd.read_excel(xls, 'Customers')

# Warehouse Fixed Cost Data
cust_dm = cust.loc[:, 'Demand (Shipments)']

# Convert to list
cust_dm = cust_dm.tolist()

In [13]:
# Supplier Variable Processing Cost per Shipment Data
wh_vpcps = wh.loc[:, 'Variable Processing Cost / Shipment']

# Convert to list
wh_vpcps = wh_vpcps.tolist()

all_wh_vpcps = []
for i in range(988):
    all_wh_vpcps.append(wh_vpcps)
wh_vpcps = all_wh_vpcps

### 2. Optimization Set up
Generate decision variables, constraints, objective functions and print out the results.

In [14]:
# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

In [15]:
# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

In [16]:
# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= 1000000 * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= 1000000 * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= 1000000 * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= 1000000 * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

In [17]:
# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# VC S-WH
# sum((x[i,j] * (s_vcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J)
# VC WH-C
# sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K)
# FC S-WH
# sum(s[i] * s_fc[i-1] for i in set_I)
# FC WH-C
# sum(w[j] * wh_fc[j-1] for j in set_J)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()

docplex.mp.solution.SolveSolution(obj=1.60374e+07,values={s(2):1,s(5):1,..

In [18]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
2,1.0
5,1.0


In [19]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(2, 5)",127.0
"(5, 1)",98.0
"(5, 2)",94.0
"(5, 3)",64.0
"(5, 6)",85.0


In [20]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
3,1.0
5,1.0
6,1.0


In [21]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",4000.0
"(1, 6)",3806.0
"(1, 9)",3694.0
"(1, 10)",3658.0
"(1, 19)",3346.0
...,...
"(6, 965)",100.0
"(6, 966)",100.0
"(6, 967)",100.0
"(6, 973)",100.0


### Pricing per parcel

In [22]:
revenue = opt_model.solution.get_objective_value() / 0.8
price = round(revenue / sum(cust_dm), 2)
price

42.86

In [23]:
# per bottle
round(price/24,2)

1.79

In [24]:
profit = revenue - opt_model.solution.get_objective_value()
profit

4009357.914999943

### 3. Sensitivity Analysis

##### 1.1 Fixed supplier/manufacturing decrease by 50% 

In [25]:
# new constraint
new_s_fc = [i*0.5 for i in s_fc]


# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * new_s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=1.55274e+07,values={s(2):1,s(5):1,..

In [26]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
2,1.0
5,1.0


In [27]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(2, 5)",127.0
"(5, 1)",98.0
"(5, 2)",94.0
"(5, 3)",64.0
"(5, 6)",85.0


In [28]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
3,1.0
5,1.0
6,1.0


In [29]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",4000.0
"(1, 6)",3806.0
"(1, 9)",3694.0
"(1, 10)",3658.0
"(1, 19)",3346.0
...,...
"(6, 965)",100.0
"(6, 966)",100.0
"(6, 967)",100.0
"(6, 973)",100.0


##### 1.2 Fixed warehouse/distribution increase by 50%

In [30]:
# new constraint
new_wh_fc = [i*1.5 for i in wh_fc]


# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * new_wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=1.65529e+07,values={s(2):1,s(5):1,..

In [31]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
2,1.0
5,1.0


In [32]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(2, 5)",131.0
"(5, 1)",98.0
"(5, 2)",135.0
"(5, 6)",104.0


In [33]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
5,1.0
6,1.0


In [34]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",4000.0
"(1, 6)",3806.0
"(1, 9)",3694.0
"(1, 10)",3658.0
"(1, 19)",3346.0
...,...
"(6, 966)",100.0
"(6, 967)",100.0
"(6, 973)",100.0
"(6, 979)",100.0


##### 1.3 Inbound cost decrease by 50%


In [35]:
# new constraint
inbound = s_vpcps+s_wh_r_matrix
new_inbound = inbound * 0.5


# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * new_inbound[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=1.28738e+07,values={s(5):1,w(1):1,..

In [36]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
5,1.0


In [37]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(5, 1)",99.0
"(5, 2)",92.0
"(5, 3)",73.0
"(5, 5)",125.0
"(5, 6)",79.0


In [38]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
3,1.0
5,1.0
6,1.0


In [39]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",4000.0
"(1, 6)",3806.0
"(1, 9)",3694.0
"(1, 10)",3658.0
"(1, 19)",3346.0
...,...
"(6, 965)",100.0
"(6, 966)",100.0
"(6, 967)",100.0
"(6, 973)",100.0


##### 1.4 Outbound costs decrease by 50%

In [40]:
# new constraint
outbound = wh_vpcps + wh_c_r_matrix
new_outbound = outbound * 0.5


# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * new_outbound[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=1.17248e+07,values={s(2):1,s(5):1,..

In [41]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
2,1.0
5,1.0


In [42]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(2, 5)",141.0
"(5, 7)",327.0


In [43]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
5,1.0
7,1.0


In [44]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(5, 2)",3960.0
"(5, 11)",3622.0
"(5, 12)",3586.0
"(5, 13)",3552.0
"(5, 15)",3482.0
...,...
"(7, 981)",100.0
"(7, 983)",100.0
"(7, 984)",100.0
"(7, 985)",100.0


##### 1.5 Fixed manufacturing decreased by 50% & fixed distribution increased by 50%

In [45]:
# new constraint
new_s_fc = [i*0.5 for i in s_fc]
new_wh_fc = [i*1.5 for i in wh_fc]


# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * new_s_fc[i-1] for i in set_I) + sum(w[j] * new_wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=1.60429e+07,values={s(2):1,s(5):1,..

In [46]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
2,1.0
5,1.0


In [47]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(2, 5)",131.0
"(5, 1)",98.0
"(5, 2)",135.0
"(5, 6)",104.0


In [48]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
5,1.0
6,1.0


In [49]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",4000.0
"(1, 6)",3806.0
"(1, 9)",3694.0
"(1, 10)",3658.0
"(1, 19)",3346.0
...,...
"(6, 966)",100.0
"(6, 967)",100.0
"(6, 973)",100.0
"(6, 979)",100.0


##### 2 The company estimates sales will grow by 25% a year, what does the optimal network looks like in year 5?

In [50]:
# new constraint
new_cust_dm = [np.ceil(i*(1.25)**4) for i in cust_dm]


# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(new_cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(new_cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == new_cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1
    
# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()

docplex.mp.solution.SolveSolution(obj=3.57976e+07,values={s(3):1,s(5):1,..

In [51]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
3,1.0
5,1.0


In [52]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(3, 5)",188.0
"(3, 9)",124.0
"(5, 1)",241.0
"(5, 2)",229.0
"(5, 3)",154.0
"(5, 6)",207.0


In [53]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
3,1.0
5,1.0
6,1.0
9,1.0


In [54]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",9766.0
"(1, 6)",9292.0
"(1, 9)",9019.0
"(1, 10)",8931.0
"(1, 19)",8169.0
...,...
"(9, 912)",245.0
"(9, 925)",245.0
"(9, 927)",245.0
"(9, 945)",245.0


##### 3 Growth in Texas (all cities) is expected to be double that of other markets (ie 50% per year in Texas versus 25% per year), does this impact the network 5 years from now?

In [55]:
new_cust = cust
for i in range(len(new_cust)):
    if new_cust.iloc[i].State == "TX":
        new_cust.loc[i,"demand"] = np.ceil(new_cust.iloc[i]["Demand (Shipments)"]*(1.50)**4)
    else:
        new_cust.loc[i,"demand"] = np.ceil(new_cust.iloc[i]["Demand (Shipments)"]*(1.25)**4)
new_cust

# new constraint
new_cust_dm = [i for i in new_cust.demand]

In [56]:
# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(new_cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(new_cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == new_cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=3.89323e+07,values={s(3):1,s(5):1,..

In [57]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
3,1.0
5,1.0


In [58]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(3, 5)",188.0
"(3, 9)",124.0
"(5, 1)",241.0
"(5, 2)",230.0
"(5, 3)",259.0
"(5, 6)",207.0


In [59]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
3,1.0
5,1.0
6,1.0
9,1.0


In [60]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",9766.0
"(1, 6)",9292.0
"(1, 9)",9019.0
"(1, 10)",8931.0
"(1, 19)",8169.0
...,...
"(9, 912)",245.0
"(9, 925)",245.0
"(9, 927)",245.0
"(9, 945)",245.0


##### 4 The company is considering charging for shipping. If they forecast losing 50% of customers, what is the impact to overall profits (note lost revenue but with less expenses)?

In [61]:
def solve_optimal(new_cust_dm):
    # Import the Model class from docplex.mp
    from docplex.mp.model import Model
    opt_model = Model(name='DHL Supply Chain Network')

    # Model Decision Variables
    set_I = range(1, 11)
    set_J = range(1, 12)
    set_K = range(1, len(new_cust_dm))

    # Supplier selection s(i)  
    s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

    # Warehouse selection: w(j)
    w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

    # Quantity of truckloads shipped from supplier i to warehouse j
    x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

    # Warehouse to customer shipment amount (parcel amount): y(j,k)
    y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

    # Constraints
    constraints = {}

    # S-WH shipment >= 0
    count = 0
    for j in set_J:
        for i in set_I:
            constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
            count += 1

    # WH-C shipment >= 0
    for j in set_J:
        for k in set_K:
            constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
            count += 1

    # WH: sum(y(j,k)) <= sum(D) * wj
    for j in set_J:
        constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(new_cust_dm) * w[j], ctname="constraint_{0}".format(count))})
        count += 1

    # S: sum(x(i,j)) <= sum(D) * si
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(new_cust_dm) * s[i], ctname="constraint_{0}".format(count))})
        count += 1

    # Warehouse inbound > outbound
    for j in set_J:
        constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
        count += 1

    # Customer demand
    for k in set_K:   
        constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == new_cust_dm[k-1], ctname="constraint_{0}".format(count))})
        count += 1

    # Objective function
    func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (wh_vpcps+wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
    objective = opt_model.sum(func)

    # For minimization
    opt_model.minimize(objective)

    # Solve the model
    opt_model.solve()
    
    return opt_model.solution.get_objective_value()


In [62]:
cost = []
demand = []
seed = [0, 10, 200, 1500, 3000]

for i in range(0,len(seed)):
    new_cust = cust.sample(frac = 0.5, random_state = seed[i])
    new_cust_dm = np.round(new_cust.loc[:, 'Demand (Shipments)'] * 0.5)
    new_cust_dm = new_cust_dm.tolist()
    demand.append(sum(new_cust_dm))
    cost.append(solve_optimal(new_cust_dm))

In [63]:
avg_cost = np.mean(cost)
avg_demand = np.mean(demand)
new_profit = price*avg_demand - avg_cost

profit_diff = new_profit - profit
profit_diff

-3928976.663000117

##### 5 A new option is available, where a customer can receive shipments with twice the standard number of bottles.  Under this option customers will receive ½ the number of overall shipments versus before, but both shipping and order processing will increase by 50%.  How will this influence the network if implemented across all shipments?

In [64]:
new_cust = cust
new_cust_dm = np.round(new_cust.loc[:, 'Demand (Shipments)'] * 0.5)
new_cust_dm = new_cust_dm.tolist()

new_wh_c_r_matrix = wh_c_r_matrix * 1.5

new_wh_vpcps = wh.loc[:, 'Variable Processing Cost / Shipment'] * 1.5
new_wh_vpcps = new_wh_vpcps.tolist()
new_all_wh_vpcps = []
for i in range(988):
    new_all_wh_vpcps.append(new_wh_vpcps)
new_wh_vpcps = new_all_wh_vpcps

In [65]:
# Import the Model class from docplex.mp
from docplex.mp.model import Model
opt_model = Model(name='DHL Supply Chain Network')

# Model Decision Variables
set_I = range(1, 11)
set_J = range(1, 12)
set_K = range(1, 989)

# Supplier selection s(i)  
s  = {(i): opt_model.binary_var(name="s({0})".format(i)) for i in set_I }

# Warehouse selection: w(j)
w  = {(j): opt_model.binary_var(name="w({0})".format(j)) for j in set_J }

# Quantity of truckloads shipped from supplier i to warehouse j
x  = {(i,j): opt_model.integer_var(name="x({0},{1}))".format(i,j)) for i in set_I for j in set_J}

# Warehouse to customer shipment amount (parcel amount): y(j,k)
y  = {(j,k): opt_model.integer_var(name="y({0},{1}))".format(j,k)) for j in set_J for k in set_K }

# Constraints
constraints = {}

# S-WH shipment >= 0
count = 0
for j in set_J:
    for i in set_I:
        constraints.update({count : opt_model.add_constraint(ct=x[i,j] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH-C shipment >= 0
for j in set_J:
    for k in set_K:
        constraints.update({count : opt_model.add_constraint(ct=y[j,k] >= 0, ctname="constraint_{0}".format(count))})
        count += 1
        
# WH: sum(y(j,k)) <= sum(D) * wj
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for k in set_K) <= sum(new_cust_dm) * w[j], ctname="constraint_{0}".format(count))})
    count += 1

# S: sum(x(i,j)) <= sum(D) * si
for i in set_I:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for j in set_J) <= sum(new_cust_dm) * s[i], ctname="constraint_{0}".format(count))})
    count += 1
    
# Warehouse inbound > outbound
for j in set_J:
    constraints.update({count : opt_model.add_constraint(ct=sum(x[i,j] for i in set_I) >= sum(y[j,k] for k in set_K) / 1000, ctname="constraint_{0}".format(count))})
    count += 1
            
# Customer demand
for k in set_K:   
    constraints.update({count : opt_model.add_constraint(ct=sum(y[j,k] for j in set_J) == new_cust_dm[k-1], ctname="constraint_{0}".format(count))})
    count += 1

# Objective function
func = sum((x[i,j] * (s_vpcps+s_wh_r_matrix)[i-1,j-1]) for i in set_I for j in set_J) + sum((y[j,k] * (new_wh_vpcps+new_wh_c_r_matrix)[k-1,j-1]) for j in set_J for k in set_K) + sum(s[i] * s_fc[i-1] for i in set_I) + sum(w[j] * wh_fc[j-1] for j in set_J)
objective = opt_model.sum(func)

# For minimization
opt_model.minimize(objective)

# Solve the model
opt_model.solve()


docplex.mp.solution.SolveSolution(obj=1.08916e+07,values={s(5):1,w(1):1,..

In [66]:
# Result for supplier selection
opt_df = pd.DataFrame.from_dict(s, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
5,1.0


In [67]:
# Result for supplier to warehouse shipment
opt_df = pd.DataFrame.from_dict(x, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(5, 1)",49.0
"(5, 2)",68.0
"(5, 5)",65.0
"(5, 6)",52.0


In [68]:
# Result for warehouse selection
opt_df = pd.DataFrame.from_dict(w, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
1,1.0
2,1.0
5,1.0
6,1.0


In [69]:
# Result for warehouse to customer shipment
opt_df = pd.DataFrame.from_dict(y, orient="index", columns = ["variable_object"])
sett = pd.DataFrame(opt_df["variable_object"].apply(lambda item: np.round(item.solution_value)))
sett[sett.variable_object != 0]

,variable_object
"(1, 1)",2000.0
"(1, 6)",1903.0
"(1, 9)",1847.0
"(1, 10)",1829.0
"(1, 19)",1673.0
...,...
"(6, 966)",50.0
"(6, 967)",50.0
"(6, 973)",50.0
"(6, 979)",50.0
